# 1. 免学部分
* 这一章只有14面内容，所以暂不设免学部分

# 2. 前言

* 大家在做题目前一定要先把第5章的书大概的读一下
* 写本作业，请使用 Anaconda 里的 Notebook
* 我做了一个[打包下载 Link](https://downgit.github.io/#/home?url=https://github.com/liuhui998/pe2/tree/master/05-missing), 大家下载下来解压后，使用 Notebook 打开

# 3. 学习概要
* 我会在每道题有前标明对的书页码
* 如果超纲我会提前提示，并加上资料 Link
---

知识点整理

* 

In [17]:
import pandas as pd

# 找出订单表中没有物流号的订单

* 读取本地目录下的 trackings_all.xlsx 到  trackings_all
* 找出 tracking_no 为空的数据，并输出
* 将tracking_no 为空的数据的 tracking_no 改为 "missing", is_delivered 为 0


In [18]:
trackings_all = pd.read_excel("trackings_all.xlsx")
trackings_all

,order_no,sku,quantity,tracking_no,is_delivered
0,O_2020_01,A1234,2,T_2020_1,1.0
1,O_2020_02,A1234,2,T_2020_2,0.0
2,O_2020_03,A1235,1,T_2020_3,1.0
3,O_2020_04,A1235,1,NaN,NaN


In [27]:
# 这样改不会改掉 trackings_all 里的内容
trackings_all.tracking_no.fillna("missing")

0    T_2020_1
1    T_2020_2
2    T_2020_3
3     missing
Name: tracking_no, dtype: object

In [28]:
# 现在 tracking_no 还是为空
trackings_all

,order_no,sku,quantity,tracking_no,is_delivered
0,O_2020_01,A1234,2,T_2020_1,1.0
1,O_2020_02,A1234,2,T_2020_2,0.0
2,O_2020_03,A1235,1,T_2020_3,1.0
3,O_2020_04,A1235,1,NaN,NaN


In [30]:
# 将tracking_no 为空的数据的 tracking_no 改为 "missing"
# 加入了 inplace = True, 就会直接改原始值，而不只是返回一个新的数值
# inplace = True 在后面很多函数都可以用到，大家一定记住
trackings_all.tracking_no.fillna('missing',inplace = True)
trackings_all

,order_no,sku,quantity,tracking_no,is_delivered
0,O_2020_01,A1234,2,T_2020_1,1.0
1,O_2020_02,A1234,2,T_2020_2,0.0
2,O_2020_03,A1235,1,T_2020_3,1.0
3,O_2020_04,A1235,1,missing,NaN


In [31]:
# 将tracking_no 为空的数据的  is_delivered 为 0
trackings_all.is_delivered.fillna(0,inplace = True)
trackings_all

,order_no,sku,quantity,tracking_no,is_delivered
0,O_2020_01,A1234,2,T_2020_1,1.0
1,O_2020_02,A1234,2,T_2020_2,0.0
2,O_2020_03,A1235,1,T_2020_3,1.0
3,O_2020_04,A1235,1,missing,0.0


# 处理 Amazon review 数据缺失的情况
---
* 现在有一张 SKU A1234 的 Amazon Review 表
* 字段有两列: day（日期）, star（星级）
* 现在有一些的数据是缺失的，我们想把它补上

---


# 处理缺失的客户调查数据
---
* 现在有一张客户调查表 survey
* 字段有 name(姓名),gender(性别),income(收入）

--- 
* 删掉所有 gender(性别） 为空的调查数据

* 分别计算男性 (gender=='Male') 的平均收入 

  (先对 gender 过滤, 再对 income 用 sum 求和，记得使用 skipna 参数）
  
* 分别计算男性 (gender=='Female') 的平均收入

* 使用男性平均收入，填充男性客户收入值缺失的数据 (income.isnull())

* 使用女性平均收入，填充女性客户收入值缺失的数据 (income.isnull())